In [1]:
!ls

CodeT5.ipynb  Models  data.jsonl


In [12]:
!pip install transformers --force --ignore-installed PyYAML


  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached filelock-3.6.0-py3-none-any.whl (10.0 kB)
Processing /home/harish/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9/sacremoses-0.0.53-py3-none-any.whl
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached regex-2022.4.24-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (749 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached importlib_metadata-4.11.3-py3-none-any.whl (18 kB)
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Using cached six-1

In [8]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html 


Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp37-cp37m-linux_x86_64.whl (2041.3 MB)
  Using cached https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl (23.2 MB)
  Using cached torchaudio-0.9.0-cp37-cp37m-manylinux1_x86_64.whl (1.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0


In [9]:
import torch

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [14]:
tc2code = pd.read_json('./data.jsonl',lines=True)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
print(tc2code.head())

                                            testCode  \
0  /*\n * This file was automatically generated b...   
1  /*\n * This file was automatically generated b...   
2  /*\n * This file was automatically generated b...   
3  /*\n * This file was automatically generated b...   
4  /*\n * This file was automatically generated b...   

                                          sourceCode  
0  package macaw.util;\n\nimport macaw.system.Mac...  
1  package macaw.util;\n\nimport macaw.system.Use...  
2  package macaw.util;\n\nimport java.util.regex....  
3  package macaw.util;\n\nimport java.awt.event.C...  
4  package macaw.util;\n\nimport javax.swing.*;\n...  


Random data Explorations

In [17]:
from transformers import T5Tokenizer, T5Model,T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small").to(device)
# model=T5Model.from_pretrained("Salesforce/codet5-small")
input_ids = tokenizer(
    tc2code['testCode'][0], return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer(tc2code['sourceCode'][0], return_tensors="pt").input_ids  # Batch size 1

# forward pass
# outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
# last_hidden_states = outputs.last_hidden_state

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1021 > 512). Running this sequence through the model will result in indexing errors


In [18]:
sample_outputs = model.generate(
    input_ids.to(device),
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

# outputs = model.generate(input_ids)
print("Generated docstring:", tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Generated docstring:  public}if (label == "Labellabelfalse ; if}
{
}
public voidthis {({
(;;{



In [19]:
model.cpu()
del model

Define Tokenizer and Tokenize the input and op

In [20]:
# tc2code['token_lengths']=tc2code['sourceCode'].apply(lambda x: len(tokenizer.tokenize(x)))

In [21]:
# tc2code['token_lengths_test']=tc2code['testCode'].apply(lambda x: len(tokenizer.tokenize(x)))

In [22]:
# tc2code['token_lengths'].describe()

The 75th percentile is 1766 hence we choose 1800 as the max_length

In [23]:
# tc2code['token_lengths_test'].describe()

The 75th percentile is 972 so lets go with 1766

In [24]:
def RobertaTokenizerForSeq2Seq(x):
  targetSeq=tokenizer(x,max_length=1800,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True).input_ids
    ## Regularize Cross Entropy loss
  #  target_seq_with_ignore_index=[]
  #  for seq in targetSeq:
  #     targetseq_example = [label if label != 0 else -100 for label in seq]
  #     target_seq_with_ignore_index.append(targetseq_example)
  #  return torch.Tensor(target_seq_with_ignore_index)
  targetSeq[targetSeq==0]=-100
  return targetSeq



In [25]:
def RobertaTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=1000,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

In [26]:
print(RobertaTokenizerForSeq2Seq(tc2code['sourceCode'][0]))

tensor([[   1, 5610, 5318,  ..., -100, -100, -100]])


Report ඞ

In [27]:
sample_code=tc2code.head(10)
sample_code['test_Samples']=sample_code['testCode'].apply(lambda x: tokenizer.tokenize(x))
sample_code['source_Samples']=sample_code['sourceCode'].apply(lambda x: tokenizer.tokenize(x))
sample_code.head(10)

/home/harish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/harish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,testCode,sourceCode,test_Samples,source_Samples
0,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
1,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Use...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
2,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.util.regex....,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
3,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.awt.event.C...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
4,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport javax.swing.*;\n...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
5,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.businessLa...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
6,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
7,/*\n * This file was automatically generated b...,/*\n * Created on 25-Jul-2009\n * Copyright (C...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[/*, Ċ, Ġ*, ĠCreated, Ġon, Ġ25, -, J, ul, -, 2..."
8,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
9,/*\n * This file was automatically generated b...,/*\n * Created on 25-Jul-2009\n * Copyright (C...,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[/*, Ċ, Ġ*, ĠCreated, Ġon, Ġ25, -, J, ul, -, 2..."


In [28]:
RobertaTokenizerForSeq2SeqTest(sample_code['testCode'].to_list())

{'input_ids': tensor([[    1, 20308,   203,  ...,    67,  1919,     2],
        [    1, 20308,   203,  ...,   225,  1071,     2],
        [    1, 20308,   203,  ...,     0,     0,     0],
        ...,
        [    1, 20308,   203,  ...,     0,     0,     0],
        [    1, 20308,   203,  ...,   225,  1071,     2],
        [    1, 20308,   203,  ...,   225,  1071,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

Training the models hopefully

In [29]:
X_train, X_test, y_train, y_test = train_test_split(tc2code['testCode'], tc2code['sourceCode'], test_size=0.2, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.1, shuffle=False)

In [30]:
X_train_tensors=RobertaTokenizerForSeq2SeqTest(X_train.to_list())
X_test_tensors=RobertaTokenizerForSeq2SeqTest(X_test.to_list())
X_val_tensors=RobertaTokenizerForSeq2SeqTest(X_val.to_list())

In [31]:
y_train_tensors=RobertaTokenizerForSeq2Seq(y_train.to_list())
y_test_tensors=RobertaTokenizerForSeq2Seq(y_test.to_list())
y_val_tensors=RobertaTokenizerForSeq2Seq(y_val.to_list())

In [32]:
input_ids_train = X_train_tensors['input_ids']
attention_masks_train = X_train_tensors['attention_mask']
labels_train = y_train_tensors

#validation set
input_ids_val = X_val_tensors['input_ids']
attention_masks_val = X_val_tensors['attention_mask']
labels_val = y_val_tensors


input_ids_test = X_test_tensors['input_ids']
attention_masks_test = X_test_tensors['attention_mask']
labels_test = y_test_tensors
expected_source_code=y_test.to_list()

In [33]:
expected_source_code[0]

'/*\n * Created on 14/mar/2010\n * Copyright (C) 2010 by Andrea Vacondio.\n *\n * This program is free software; you can redistribute it and/or modify it under the terms of the \n * GNU General Public License as published by the Free Software Foundation; \n * either version 2 of the License.\n * This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; \n * without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. \n * See the GNU General Public License for more details.\n * You should have received a copy of the GNU General Public License along with this program; \n * if not, write to the Free Software Foundation, Inc., \n *  59 Temple Place, Suite 330, Boston, MA 02111-1307 USA\n */\npackage org.pdfsam.guiclient.business.listeners;\n\nimport java.awt.event.InputEvent;\nimport java.awt.event.MouseWheelEvent;\nimport java.awt.event.MouseWheelListener;\n\nimport org.pdfsam.guiclient.gui.components.JPreviewImage;\n\n/**\n * 

In [34]:
import gc
gc.collect()

60

Training the model

In [47]:
t5_model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small",dropout_rate=0.5,d_model=1850,ignore_mismatched_sizes=True).to(device)
t5_model.to(device)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at Salesforce/codet5-small and are newly initialized because the shapes did not match:
- shared.weight: found shape torch.Size([32100, 512]) in the checkpoint and torch.Size([32100, 1850]) in the model instantiated
- encoder.embed_tokens.weight: found shape torch.Size([32100, 512]) in the checkpoint and torch.Size([32100, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.q.weight: found shape torch.Size([512, 512]) in the checkpoint and torch.Size([512, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.k.weight: found shape torch.Size([512, 512]) in the checkpoint and torch.Size([512, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.v.weight: found shape torch.Size([512, 512]) in the checkpoint and torch.Size([512, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.o.weight: found shape torch.Size([512, 512

T5ForConditionalGeneration(
  (shared): Embedding(32100, 1850)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1850)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1850, out_features=512, bias=False)
              (k): Linear(in_features=1850, out_features=512, bias=False)
              (v): Linear(in_features=1850, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=1850, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.5, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1850, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=1850, bias=False)
              (dropo

In [48]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                             attention_masks_val, 
                             labels_val)

# dataset_test = TensorDataset(input_ids_test, 
#                              attention_masks_test, 
#                              labels_test)

Defining a Custom test loader

In [49]:

import torch
tr=torch.randn((2,3))
print(tr)
print(tr[0])

tensor([[-0.1515, -1.5568,  1.1424],
        [ 0.6822, -0.2215,  1.3309]])
tensor([-0.1515, -1.5568,  1.1424])


In [50]:
from torch.utils import data
class CodeTCDatasetTest(data.Dataset):
    def __init__(self, testTensor,sourceTensor,testExpectedCode):
      self.testTensor=testTensor
      self.sourceTensor=sourceTensor
      self.testExpectedCode=testExpectedCode


    def __getitem__(self, index):
        return self.testTensor[index],self.sourceTensor[index],self.testExpectedCode[index]


    def __len__(self):
        return len(self.testExpectedCode)


def collate_fn(data):
    testTensorList,sourceTensorList,sourceCode = zip(*data)

    testTensors = torch.stack(testTensorList, 0)
    sourceTensors = torch.stack(sourceTensorList, 0)
      
    return testTensors, sourceTensors, sourceCode

def get_test_loader(testTensor,sourceTensors,sourceCodeList,batch_size):
    """Returns torch.utils.data.DataLoader for custom coco dataset."""
    # COCO caption dataset
    code2tctest = CodeTCDatasetTest(testTensor,sourceTensors,sourceCodeList)
    
    # Data loader for COCO dataset
    # This will return (images, captions, lengths) for each iteration.
    # images: a tensor of shape (batch_size, 3, 224, 224).
    # captions: a tensor of shape (batch_size, padded_length).
    # lengths: a list indicating valid length for each caption. length is (batch_size).
    data_loader = torch.utils.data.DataLoader(dataset=code2tctest, 
                                              batch_size=batch_size,
                                              collate_fn=collate_fn)
    return data_loader

In [57]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

#train set
dataloader_train = DataLoader(dataset_train,
                              sampler = RandomSampler(dataset_train),
                              batch_size = batch_size)

#validation set
dataloader_val = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = batch_size)

dataloader_test = get_test_loader(input_ids_test,labels_test,expected_source_code,batch_size)

In [58]:
from transformers import get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(t5_model.parameters(),
                 lr = 1e-5,
                 eps = 1e-8)
                 
epochs = 30

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = len(dataloader_train)*epochs)

In [59]:
def evaluate(dataloader_val,model):

    #evaluation mode 
    model.eval()
    
    #tracking variables
    loss_val_total = 0
    
    for batch in tqdm(dataloader_val):
        
        #load into GPU
        batch = tuple(b.to(device) for b in batch)
        
        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        loss_val_total += loss.item()

    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
            
    return loss_val_avg

In [60]:
def train_model(model):
    save_epoch=5
    validation_loss_epochs=[]
    training_loss_epochs=[]

    for epoch in tqdm(range(1, epochs+1)):
        
        model.train()
        
        loss_train_total = 0
        
        progress_bar = tqdm(dataloader_train, 
                            desc = 'Epoch {:1d}'.format(epoch), 
                            leave = False, 
                            disable = False)
        
        for batch in progress_bar:
            
            model.zero_grad() #set gradient to 0
        
            batch = tuple(b.to(device) for b in batch)
            
            inputs = {'input_ids': batch[0], 
                      'attention_mask': batch[1], 
                      'labels': batch[2]}
            
            outputs = model(**inputs) #unpack the dict straight into inputs
            
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

        if epoch%save_epoch==0:    
          torch.save(model.state_dict(), f'Models/ CodeT5_ft_epoch{epoch}.model')
        
        tqdm.write('\n Epoch {epoch}')
        
        loss_train_ave = loss_train_total / len(dataloader_train)
        tqdm.write('Training loss: {loss_train_avg}')
        
        val_loss = evaluate(dataloader_val,model)
        validation_loss_epochs.append(val_loss)
        training_loss_epochs.append(loss_train_ave)
        
        tqdm.write(f'Validation loss: {val_loss}')
        return validation_loss_epochs,training_loss_epochs,model

In [ ]:
val_losses,training_losses,t5_model=train_model(t5_model)

Epoch 1:   2%|▏         | 48/2295 [00:28<21:29,  1.74it/s, training_loss=3.388]


Epoch 1:   4%|▍         | 97/2295 [00:56<21:03,  1.74it/s, training_loss=2.911]


Epoch 1:   6%|▋         | 146/2295 [01:23<20:29,  1.75it/s, training_loss=2.794]


Epoch 1:   8%|▊         | 194/2295 [01:51<20:01,  1.75it/s, training_loss=2.679]


Epoch 1:  11%|█         | 243/2295 [02:19<19:35,  1.75it/s, training_loss=2.612]


Epoch 1:  12%|█▏        | 269/2295 [02:33<19:21,  1.74it/s, training_loss=2.544]

In [ ]:

n_epochs=len(training_losses)

epochs_arr=np.arange(start=1,stop=n_epochs+1,step=1)
import matplotlib.pyplot as plt

plt.plot(epochs_arr,val_losses,label='Validation Loss')
plt.plot(epochs_arr,training_losses,label='Training Loss')

plt.xlabel('epochs')
plt.ylabel('Loss')

plt.title('Epoch vs Loss')
plt.legend()
plt.show()

In [61]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def getPredictionsTopK(inputs,model,device):
      sample_outputs = model.generate(
        input_ids.to(device),
        do_sample=True, 
        max_length=1800, 
        top_k=900, 
        top_p=0.95, 
        num_return_sequences=3)
    # outputs = model.generate(input_ids)
      sequences=[]
      for i in range(sample_outputs.shape[0]):
        sequences.append(tokenizer.decode(sample_outputs[i], skip_special_tokens=True))
      return sequences

def getPredictionsGreedy(inputs,model,device):
      greedy_output = model.generate(input_ids.to(device), max_length=1800)
      sequences=[]
      for i in range(greedy_output.shape[0]):
        sequences.append(tokenizer.decode(greedy_output[i], skip_special_tokens=True))
      return sequences




In [ ]:
def getPredictedTexts(dataloader_test,genAlgo,model,device):
    predictions=[]
    allExpectedprogs=[]
    idx=0
    for ip,eop,estr in dataloader_test:
        print(f"iter{idx} of {len(dataloader_test)}")
        prediction=genAlgo(ip,model,device)
        predictions=predictions+prediction
        expected=[x for x in estr]
        allExpectedprogs=allExpectedprogs+expected
        idx+=1
    return allExpectedprogs,predictions

In [ ]:
allExpectedprogsGreedy,predictionsGreedy=getPredictedTexts(dataloader_test,getPredictionsGreedy,t5_model,device)

In [ ]:
predictiondf_t5 = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy)), columns =['Expected', 'predictions']) 

In [ ]:
predictiondf_t5.to_json('./predictions_codet5.jsonl',orient='records', lines=True)

In [ ]:
!ls

In [ ]:
torch.save(t5_model.state_dict(), f'Models/CodeT5_ft_final.model')